In [1]:
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.losses import MeanAbsolutePercentageError as MAPE
from random import choice,random
import tensorflow as tf
from pprint import pformat,pprint
import keras
import pickle
from keras import layers, models, regularizers, backend, utils
import keras.backend.tensorflow_backend as tfback
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline

Using TensorFlow backend.


In [2]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
TEST_RATIO = 0.2
REPEAT = 1
activations = ["sigmoid","relu"]
OPTIONS = {"first_lay" : [7,8,9,10],
           "second_lay":[7,8,9,10,11],
           "third_lay" : [7,8,9,10],
           "drop_amt":[0,0.05,0.1,0.15,0.2],
           "act": activations,
           "hyper":[0,0.003,0.01,0.03,0.1],
           "batch":[128,256,500,1000,2000,8000,20000]
          }

class option(object):
    def __init__(self):
        self.first_lay = 2**(choice(OPTIONS["first_lay"]))
        self.second_lay = 2**(choice(OPTIONS["second_lay"]))
        self.third_lay = 2**(choice(OPTIONS["third_lay"]))
        self.drop_amt = choice(OPTIONS["drop_amt"])
        self.act =  choice(OPTIONS["act"])
        self.hyper = choice(OPTIONS["hyper"])
        self.batch = choice(OPTIONS["batch"])
    def __str__(self):
        return pformat(self.__dict__)
    def get_sim(self,verbose=False):
        x = option()
        x.__dict__ = dict(self.__dict__)
        pick = choice(list(x.__dict__.keys()))
        
        new_val = choice(OPTIONS[pick])
        if verbose:
            print(f"Changed entry {pick} to {new_val}")
        x.__dict__[pick]= new_val
        if "lay" in pick:
            x.__dict__[pick] = 2**x.__dict__[pick]
        return x
        
K = keras.backend

In [4]:
mape = MAPE()

files = ["/notebooks/WECs_DataSet/Sydney_Data.csv"]
csvfile = open(files[0])
reader = csv.reader(csvfile, delimiter = ',')
rows = [[float(v) for v in row] for row in reader]
rows = np.asarray(rows, dtype='float32')

In [5]:
xs = rows[:,0:32]
ys = rows[:,32:49]
ys = ys / np.max(ys)
xs = xs / np.max(xs)
xs_train, xs_test, ys_train, ys_test = train_test_split(xs,ys,test_size = TEST_RATIO)

if REPEAT>1:#randomize indices of data...didnt help so much
    xs_train = np.reshape(xs_train,(57600,16,2))
    final_train = np.zeros((0,16,2))
    for _ in range(REPEAT):
        hold = np.take(xs_train,np.random.permutation(16),axis=1)
        final_train = np.concatenate([final_train,hold])
    ys_train = np.tile(ys_train.transpose(),REPEAT).transpose()
    xs_train = np.reshape(final_train,(57600*REPEAT,32))
print(xs_train.shape,ys_train.shape)

(57600, 32) (57600, 17)


In [6]:
def get_error(model):
    ys_predictions = model.predict(xs_test)
    mape = tf.keras.losses.MeanAbsolutePercentageError()
    error = mape(ys_predictions, ys_test).numpy()
    return error

In [7]:
def test_options(opt,return_mod = False,patience=100,verbose=0):
    def custom_loss(y_true,y_pred):
      return K.mean(K.square(y_pred - y_true), axis=-1) + opt.hyper * K.mean(K.square(K.sum(y_pred[:,0:16], axis=1) - y_pred[:,16]), axis=-1)

    model = models.Sequential()
    model.add(layers.Dense(opt.first_lay, input_dim=32, activation = opt.act))
    model.add(layers.Dropout(opt.drop_amt))
    model.add(layers.Dense(opt.second_lay, activation = opt.act))
    model.add(layers.Dense(opt.third_lay, activation = opt.act))
    model.add(layers.Dense(17, activation = 'linear'))
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience)

    model.compile(keras.optimizers.Adam(learning_rate=0.0003),custom_loss,
                  metrics=['mean_absolute_percentage_error'])
    if verbose!=0:
        model.summary()
    loss_hist = model.fit(xs_train, ys_train, epochs = 30000, shuffle=True,verbose=verbose,
                             validation_data = (xs_test, ys_test), batch_size=opt.batch, callbacks=[callback])
    error = get_error(model)
    if return_mod:
        return error,model
    return error

In [ ]:
while True:
    with open("list.pickle","rb") as  f:
        lst = pickle.load(f)

    #very simple genetic algorithm
    new_opt = option()
    if random()<0.2:
        print("picking fresh option:")
        print(f"FRESH NEW OPTION:{new_opt}")
    else:
        old_opt,score = choice(lst[:1])
        new_opt = old_opt.get_sim(verbose=True) 
        
        print(f"Making from old option {old_opt} with score {score} a new option {new_opt}")
        
    new_opt.first_lay = 2**10
    new_opt.second_lay = 2**11
    new_opt.third_lay = 2**10
    print(new_opt)
    score,model = test_options(new_opt,return_mod = True, patience = 100, verbose = 1)
    lst.append((new_opt,score))
    print(lst[-1][1])
    print()
    lst = sorted(lst,key = lambda x: x[1])
    if score == lst[0][1]:
        print("\n"*5+"NEW BEST!!!\n"*3+"Now saving model....\n"*5)
        model.save("model.h5")
    print(lst[0][1],lst[0][0])
    print([x[1] for x in lst])
    with open("list.pickle","wb") as f:
        pickle.dump(lst,f)
        

Changed entry hyper to 0
Making from old option {'act': 'relu',
 'batch': 2000,
 'drop_amt': 0.1,
 'first_lay': 512,
 'hyper': 0.003,
 'second_lay': 512,
 'third_lay': 256} with score 2.152113914489746 a new option {'act': 'relu',
 'batch': 2000,
 'drop_amt': 0.1,
 'first_lay': 512,
 'hyper': 0,
 'second_lay': 512,
 'third_lay': 256}
{'act': 'relu',
 'batch': 2000,
 'drop_amt': 0.1,
 'first_lay': 1024,
 'hyper': 0,
 'second_lay': 2048,
 'third_lay': 1024}
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              33792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              2099200   
________________________________________________________

Epoch 34/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.8308e-05 - mean_absolute_percentage_error: 4.8529 - val_loss: 1.7898e-05 - val_mean_absolute_percentage_error: 3.6796
Epoch 35/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.7642e-05 - mean_absolute_percentage_error: 4.7765 - val_loss: 1.4961e-05 - val_mean_absolute_percentage_error: 3.6115
Epoch 36/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.7452e-05 - mean_absolute_percentage_error: 4.7115 - val_loss: 1.2507e-05 - val_mean_absolute_percentage_error: 3.5655
Epoch 37/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.7889e-05 - mean_absolute_percentage_error: 4.6655 - val_loss: 2.3741e-05 - val_mean_absolute_percentage_error: 3.7290
Epoch 38/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.6534e-05 - mean_absolute_percentage_error: 4.5864 - val_loss: 1.9617e-05 - val_mean_absolute_percentage

57600/57600 [==============================] - 1s 24us/step - loss: 1.0687e-05 - mean_absolute_percentage_error: 3.4469 - val_loss: 9.9440e-06 - val_mean_absolute_percentage_error: 3.0372
Epoch 75/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.1106e-05 - mean_absolute_percentage_error: 3.4427 - val_loss: 1.0581e-05 - val_mean_absolute_percentage_error: 3.0316
Epoch 76/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.0724e-05 - mean_absolute_percentage_error: 3.4151 - val_loss: 9.9091e-06 - val_mean_absolute_percentage_error: 3.0049
Epoch 77/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.0048e-05 - mean_absolute_percentage_error: 3.3885 - val_loss: 9.3770e-06 - val_mean_absolute_percentage_error: 3.0148
Epoch 78/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.0917e-05 - mean_absolute_percentage_error: 3.4012 - val_loss: 8.3289e-06 - val_mean_absolute_percentage_error: 2.9671


57600/57600 [==============================] - 1s 24us/step - loss: 8.8457e-06 - mean_absolute_percentage_error: 3.1004 - val_loss: 1.6290e-05 - val_mean_absolute_percentage_error: 3.1704
Epoch 115/30000
57600/57600 [==============================] - 1s 24us/step - loss: 1.0867e-05 - mean_absolute_percentage_error: 3.1709 - val_loss: 8.2182e-06 - val_mean_absolute_percentage_error: 2.9453
Epoch 116/30000
57600/57600 [==============================] - 1s 24us/step - loss: 9.0498e-06 - mean_absolute_percentage_error: 3.1036 - val_loss: 1.0458e-05 - val_mean_absolute_percentage_error: 3.0205
Epoch 117/30000
57600/57600 [==============================] - 1s 24us/step - loss: 9.8186e-06 - mean_absolute_percentage_error: 3.1255 - val_loss: 7.6911e-06 - val_mean_absolute_percentage_error: 2.8474
Epoch 118/30000
57600/57600 [==============================] - 1s 24us/step - loss: 9.4958e-06 - mean_absolute_percentage_error: 3.1110 - val_loss: 1.5540e-05 - val_mean_absolute_percentage_error: 3.1

57600/57600 [==============================] - 1s 24us/step - loss: 8.5417e-06 - mean_absolute_percentage_error: 2.9982 - val_loss: 7.9006e-06 - val_mean_absolute_percentage_error: 2.8440
Epoch 155/30000
57600/57600 [==============================] - 1s 24us/step - loss: 8.9940e-06 - mean_absolute_percentage_error: 3.0191 - val_loss: 7.3772e-06 - val_mean_absolute_percentage_error: 2.8233
Epoch 156/30000
57600/57600 [==============================] - 1s 24us/step - loss: 7.8710e-06 - mean_absolute_percentage_error: 2.9594 - val_loss: 8.2302e-06 - val_mean_absolute_percentage_error: 2.8696
Epoch 157/30000
57600/57600 [==============================] - 1s 24us/step - loss: 7.9890e-06 - mean_absolute_percentage_error: 2.9708 - val_loss: 7.8130e-06 - val_mean_absolute_percentage_error: 2.8437
Epoch 158/30000
57600/57600 [==============================] - 1s 24us/step - loss: 7.8792e-06 - mean_absolute_percentage_error: 2.9560 - val_loss: 1.0183e-05 - val_mean_absolute_percentage_error: 2.8

...

Epoch 4434/30000
57600/57600 [==============================] - 2s 26us/step - loss: 1.2386e-06 - mean_absolute_percentage_error: 1.2430 - val_loss: 4.0950e-06 - val_mean_absolute_percentage_error: 1.7233
Epoch 4435/30000
57600/57600 [==============================] - 2s 26us/step - loss: 1.2294e-06 - mean_absolute_percentage_error: 1.2393 - val_loss: 4.1752e-06 - val_mean_absolute_percentage_error: 1.7378
Epoch 4436/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2648e-06 - mean_absolute_percentage_error: 1.2531 - val_loss: 4.1260e-06 - val_mean_absolute_percentage_error: 1.7329
Epoch 4437/30000
57600/57600 [==============================] - 1s 26us/step - loss: 1.2432e-06 - mean_absolute_percentage_error: 1.2483 - val_loss: 4.0518e-06 - val_mean_absolute_percentage_error: 1.7590
Epoch 4438/30000
57600/57600 [==============================] - 2s 27us/step - loss: 1.2388e-06 - mean_absolute_percentage_error: 1.2550 - val_loss: 4.1321e-06 - val_mean_absolute_

Epoch 4474/30000
57600/57600 [==============================] - 2s 26us/step - loss: 1.2596e-06 - mean_absolute_percentage_error: 1.2501 - val_loss: 4.0129e-06 - val_mean_absolute_percentage_error: 1.7245
Epoch 4475/30000
57600/57600 [==============================] - 2s 26us/step - loss: 1.2653e-06 - mean_absolute_percentage_error: 1.2530 - val_loss: 4.0717e-06 - val_mean_absolute_percentage_error: 1.7216
Epoch 4476/30000
57600/57600 [==============================] - 1s 26us/step - loss: 1.2149e-06 - mean_absolute_percentage_error: 1.2388 - val_loss: 4.0155e-06 - val_mean_absolute_percentage_error: 1.7218
Epoch 4477/30000
57600/57600 [==============================] - 2s 26us/step - loss: 1.2277e-06 - mean_absolute_percentage_error: 1.2435 - val_loss: 4.0348e-06 - val_mean_absolute_percentage_error: 1.7356
Epoch 4478/30000
57600/57600 [==============================] - 1s 26us/step - loss: 1.2121e-06 - mean_absolute_percentage_error: 1.2445 - val_loss: 4.1687e-06 - val_mean_absolute_

Epoch 4514/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2144e-06 - mean_absolute_percentage_error: 1.2383 - val_loss: 4.0740e-06 - val_mean_absolute_percentage_error: 1.7284
Epoch 4515/30000
57600/57600 [==============================] - 1s 26us/step - loss: 1.2146e-06 - mean_absolute_percentage_error: 1.2446 - val_loss: 4.3266e-06 - val_mean_absolute_percentage_error: 1.7637
Epoch 4516/30000
57600/57600 [==============================] - 2s 27us/step - loss: 1.2161e-06 - mean_absolute_percentage_error: 1.2448 - val_loss: 4.1847e-06 - val_mean_absolute_percentage_error: 1.7317
Epoch 4517/30000
57600/57600 [==============================] - 2s 27us/step - loss: 1.2257e-06 - mean_absolute_percentage_error: 1.2404 - val_loss: 4.0192e-06 - val_mean_absolute_percentage_error: 1.7148
Epoch 4518/30000
57600/57600 [==============================] - 2s 26us/step - loss: 1.2246e-06 - mean_absolute_percentage_error: 1.2426 - val_loss: 4.0191e-06 - val_mean_absolute_

Epoch 4554/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2442e-06 - mean_absolute_percentage_error: 1.2506 - val_loss: 4.1632e-06 - val_mean_absolute_percentage_error: 1.7416
Epoch 4555/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2106e-06 - mean_absolute_percentage_error: 1.2416 - val_loss: 4.0369e-06 - val_mean_absolute_percentage_error: 1.7387
Epoch 4556/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2127e-06 - mean_absolute_percentage_error: 1.2370 - val_loss: 4.0233e-06 - val_mean_absolute_percentage_error: 1.7138
Epoch 4557/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2093e-06 - mean_absolute_percentage_error: 1.2344 - val_loss: 4.1207e-06 - val_mean_absolute_percentage_error: 1.7219
Epoch 4558/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2227e-06 - mean_absolute_percentage_error: 1.2367 - val_loss: 4.1047e-06 - val_mean_absolute_

Epoch 4594/30000
57600/57600 [==============================] - 1s 25us/step - loss: 1.2666e-06 - mean_absolute_percentage_error: 1.2358 - val_loss: 3.9858e-06 - val_mean_absolute_percentage_error: 1.7045
Epoch 4595/30000
20000/57600 [=========>....................] - ETA: 0s - loss: 1.2580e-06 - mean_absolute_percentage_error: 1.2487

In [ ]:
for key in OPTIONS.keys():
    print(key+"\t",[x.__dict__[key] for x,_ in lst])

In [ ]:
with open("list.pickle2","wb") as f:
        if input("Are you sure?")=="yes":
            pickle.dump([],f) 

In [ ]:
with open("list.pickle","rb") as  f:
        lst = pickle.load(f)
best_opt = lst[0][0]

def custom_loss(y_true,y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1) + best_opt.hyper * K.mean(K.square(K.sum(y_pred[:,0:16], axis=1) - y_pred[:,16]), axis=-1)
model = models.load_model('model.h5',custom_objects={'custom_loss': custom_loss})

def plot_errors(models):
    maxx,minn = np.max(ys_test[:,-1]),np.min(ys_test[:,-1])
    for model in models:
        ys_predictions = model.predict(xs_test)
        plt.scatter(ys_predictions[:,-1], ys_test[:,-1], alpha = 0.73, s = 0.01)
        maxx = max(np.max(ys_predictions[:,-1]),maxx)
        minn = min(np.min(ys_predictions[:,-1]),minn)
        print(minn,maxx)
    t = [minn,maxx]
    plt.axis(t+t)
    plt.plot(t,t,'r--')
    plt.show()
    
plot_errors([LinearRegression().fit(xs_train,ys_train),model])

In [ ]:
get_error(model)